In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from flowermd.base import Pack, Simulation
from flowermd.library import PPS, BeadSpring
from flowermd.utils import get_target_box_mass_density
import unyt as u

/Users/cheukhanglau/miniconda3/envs/flowermd/lib/python3.11/site-packages/mdtraj/formats/__init__.py:13: DeprecationWarning: 'xdrlib' is deprecated and slated for removal in Python 3.13
  from mdtraj.formats.trr import TRRTrajectoryFile


In [3]:
pps_mol = PPS(num_mols=20, lengths=60)

In [4]:
pps_mol.coarse_grain(beads={"A": "c1ccc(S)cc1"})

In [5]:
    ff = BeadSpring(
    r_cut=2.5,
    beads={
        "A": dict(epsilon=1, sigma=10),
    },
    bonds={
        "A-A": dict(r0=0.64, k=500),
    },
    angles={"A-A-A": dict(t0=2.8, k=50)},
)

In [6]:
ff.hoomd_forces

In [7]:
cg_system = Pack(molecules=pps_mol, density=1, edge=.2, overlap=.2,packing_expand_factor=5)

In [8]:
gsd_path='e1s10.gsd'
cg_sim = Simulation(
    initial_state=cg_system.hoomd_snapshot,
    forcefield=ff.hoomd_forces,
    gsd_write_freq=int(2e5/10),
    gsd_file_name=gsd_path,
    reference_values={"length": 1*u.nm, "mass": 1*u.Unit("amu"), "energy":1*u.Unit("kJ/mol")}
)

Initializing simulation state from a gsd.hoomd.Frame.


In [9]:
target_box = get_target_box_mass_density(density=0.5*u.Unit("g/cm**3"), mass=cg_system.mass.to("g"))
print(target_box)

[7.55471261e-07 7.55471261e-07 7.55471261e-07] cm


In [10]:
cg_sim.run_NVT(n_steps=2e5, kT=3.0, tau_kt=1.0)

RuntimeError: Particle with unique tag 1199 is no longer in the simulation box.

Cartesian coordinates: 
x: -3.22875e+08 y: -2.54805e+07 z: 2.50723e+08
Fractional coordinates: 
f.x: -1.07694e+07 f.y: -849891 f.z: 8.36275e+06
Local box lo: (-14.9904, -14.9904, -14.9904)
          hi: (14.9904, 14.9904, 14.9904)


In [ ]:
cg_sim.box_lengths

In [ ]:
target_box

In [ ]:
cg_sim.flush_writers()